In [ ]:
import tensorflow as tf
import random
import numpy as np
from tensorflow.keras import backend as K

In [ ]:
( x_train , y_train ) , ( x_test , y_test ) = tf.keras.datasets.mnist.load_data()

In [ ]:
def pair_imgs(xs , ys , nn):
    img_pairs =[]
    lables = []

    for i in range(0 , nn):
        a = random.randint(0, xs.shape[0] - 1)
        b = random.randint(0, xs.shape[0] - 1)
        img_pairs.append([xs[a] , xs[b]])
        if ys[a] == ys[b] : lables.append(1)
        else : lables.append(0)


    return np.array(img_pairs ) , np.array(lables , dtype=np.float32)

In [ ]:
train_img_pairs , train_lables = pair_imgs(x_train , y_train , 60000)
test_img_pairs , test_lables = pair_imgs(x_train , y_train , 10000)

In [ ]:
train_img_pairs.shape , train_lables.shape , test_img_pairs.shape , test_lables.shape

((60000, 2, 28, 28), (60000,), (10000, 2, 28, 28), (10000,))

In [ ]:
c = 0
for i in train_lables :
    if i == 1 : c += 1
print(c)

5939


# **Model**

In [ ]:
def Encoder(Shapes = (28,28, 1)):
    input = tf.keras.Input(shape=Shapes)
    x = tf.keras.layers.BatchNormalization()(input)
    x = tf.keras.layers.Conv2D(16 , (3,3) , activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(32 , (3,3) , activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(64 , (3,3) , activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(16 , activation = "relu")(x)

    return tf.keras.models.Model( inputs = input , outputs = x)

In [ ]:
encoder = Encoder()
encoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_13 (Conv2D)          (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 13, 13, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 5, 5, 32)         0   

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
def simiesNet(encoder , Shapes , euclidean_distance , eucl_dist_output_shape):
    input1 = tf.keras.Input(shape=Shapes)
    input2 = tf.keras.Input(shape=Shapes)

    encoderModel = encoder(Shapes)

    x1 = encoderModel(input1)
    x2 = encoderModel(input2)

    x = tf.keras.layers.Lambda(euclidean_distance, name="output_layer", output_shape=eucl_dist_output_shape)([x1, x2])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(1 , activation = "relu")(x)

    return tf.keras.models.Model( inputs = [input1 , input2] , outputs = x)


In [ ]:
def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
    return contrastive_loss

In [ ]:
model = simiesNet(Encoder , (28,28, 1) , euclidean_distance , eucl_dist_output_shape)
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 model_13 (Functional)          (None, 16)           24340       ['input_20[0][0]',               
                                                                  'input_21[0][0]']               
                                                                                                  
 output_layer (Lambda)          (None, 1)            0           ['model_13[0][0]',        

In [ ]:
rms = tf.keras.optimizers.RMSprop()
model.compile(loss=contrastive_loss_with_margin(margin=1), optimizer=rms)
history = model.fit([train_img_pairs[:,0], train_img_pairs[:,1]], train_lables, epochs=20, batch_size=128, validation_data=([test_img_pairs[:,0], test_img_pairs[:,1]], test_lables))

Epoch 1/20
469/469 [==============================] - 75s 158ms/step - loss: 0.2758 - val_loss: 0.1386
Epoch 2/20
469/469 [==============================] - 74s 157ms/step - loss: 0.1299 - val_loss: 0.1129
Epoch 3/20
469/469 [==============================] - 71s 151ms/step - loss: 0.1142 - val_loss: 0.0937
Epoch 4/20
469/469 [==============================] - 71s 150ms/step - loss: 0.0901 - val_loss: 0.0914
Epoch 5/20
469/469 [==============================] - 72s 154ms/step - loss: 0.0893 - val_loss: 0.0913
Epoch 6/20
469/469 [==============================] - 72s 154ms/step - loss: 0.0892 - val_loss: 0.0914
Epoch 7/20
469/469 [==============================] - 69s 148ms/step - loss: 0.0892 - val_loss: 0.0912
Epoch 8/20
469/469 [==============================] - 74s 159ms/step - loss: 0.0891 - val_loss: 0.0912
Epoch 9/20
469/469 [==============================] - 70s 149ms/step - loss: 0.0890 - val_loss: 0.0913
Epoch 10/20
469/469 [==============================] - 69s 148ms/step - l